In [2]:
# Contar los pares PAIS_AUTOR, PAIS_EJEMPLAR y ver cuales son las "rutas" (pares, siempre q ambos paises sean diferentes) mas frecuentes

In [3]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Préstamos

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 19.1 s, sys: 1.2 s, total: 20.3 s
Wall time: 20.3 s


# Autores

In [5]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
}

# autores_df = pd.read_csv('autores3.csv', header=0, dtype=dtypes)
autores_df = pd.read_csv('autores.csv', header=0, dtype=dtypes)
autores_df = autores_df.fillna('')

CPU times: user 5.07 s, sys: 120 ms, total: 5.19 s
Wall time: 5.19 s


In [6]:
autores_df.shape

(3054360, 4)

In [7]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


## Limpiando datos

In [8]:
autores_df = autores_df[
    (autores_df['AUTOR'] != 'DESCONOCIDO') &
    (autores_df['TITULO'] != 'DESCONOCIDO') &
    (autores_df['PAIS_AUTOR'] != 'DESCONOCIDO') &
    (autores_df['PAIS_EJEMPLAR'] != 'DESCONOCIDO')
]

In [9]:
autores_df.shape

(931392, 4)

In [10]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
10,Adivina en qué pais ...,Spain,Martina Badstuber,Germany
11,Adivina en qué pais ...,Spain,Martina Badstuber,Germany
12,Adivina en qué pais ...,Spain,Martina Badstuber,Germany
19,Humo,Spain,Antón Fortes Torres,Spain
20,Humo,Spain,Antón Fortes Torres,Spain


In [11]:
autores_df.drop_duplicates(['AUTOR', 'TITULO'], inplace=True)

In [12]:
autores_df.shape

(70687, 4)

In [13]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
10,Adivina en qué pais ...,Spain,Martina Badstuber,Germany
19,Humo,Spain,Antón Fortes Torres,Spain
21,Galleta para perros,Spain,Helen Cooper,United Kingdom
25,Autobio,Spain,Cyril Pedrosa,France
58,"Félix, : el coleccionista de miedos",Spain,Fina Casalderrey,Spain


## Resultados

In [14]:
autores_df['TITULO'].nunique()

68248

In [15]:
autores_df['TITULO'].value_counts()

Obras completas                                                 95
Antología poética                                               73
Antología                                                       66
Poemas                                                          58
Cuentos                                                         48
Cuentos completos                                               43
Poesía                                                          43
Obras escogidas                                                 33
Obras selectas                                                  30
Obras                                                           30
Obra poética                                                    28
Teatro                                                          28
Poesía completa                                                 27
Antología personal                                              23
Poesías                                                       

In [16]:
autores_df['PAIS_EJEMPLAR'].nunique()

127

In [17]:
autores_df['PAIS_EJEMPLAR'].value_counts()

Spain                                 34706
Colombia                               8751
Argentina                              6760
Mexico                                 6208
United States                          3700
France                                 1850
England                                1641
Venezuela                              1036
New York (State)                        829
Chile                                   727
Cuba                                    427
Brazil                                  425
United Kingdom                          414
Italy                                   372
Peru                                    359
Germany                                 283
Ecuador                                 266
California                              150
Canada                                  137
Uruguay                                 130
Switzerland                             113
Austria                                 108
Japan                           

In [18]:
autores_df['AUTOR'].nunique()

21055

In [19]:
autores_df['AUTOR'].value_counts()

William Shakespeare        230
Gabriel García Márquez     177
Isaac Asimov               164
Karl Marx                  160
Jorge Luis Borges          144
Oscar Wilde                130
Agatha Christie            129
Johann Sebastian Bach      117
Edgar Allan Poe            116
Ludwig van Beethoven       110
Noam Chomsky               110
Jordi Sierra i Fabra       108
Georges Simenon            106
Charles Dickens            105
Stefan Zweig               105
Pablo Neruda               103
Jules Verne                102
Mario Benedetti            101
Federico García Lorca       95
Arthur Conan Doyle          95
Fernando Pessoa             93
Honoré de Balzac            93
Sigmund Freud               92
Bertrand Russell            89
Julio Cortázar              88
Hans Christian Andersen     88
Franz Kafka                 86
Mario Vargas Llosa          86
José Martí                  85
Martin Heidegger            84
                          ... 
Paul Goldstein               1
Johnny B

In [20]:
autores_df['PAIS_AUTOR'].nunique()

315

In [21]:
autores_df['PAIS_AUTOR'].value_counts()

United States of America                       8758
United Kingdom                                 8197
Spain                                          8094
France                                         7272
Colombia                                       4412
Italy                                          3423
Germany                                        3143
Argentina                                      3071
Mexico                                         2119
Chile                                          1102
Austria                                         995
Belgium                                         889
Brazil                                          838
Switzerland                                     803
Cuba                                            798
Canada                                          797
United States                                   749
Venezuela                                       734
Ireland                                         702
Uruguay     

In [34]:
len(autores_df['PAIS_AUTOR'].value_counts())

315

In [33]:
len(autores_df['PAIS_AUTOR'].value_counts()[lambda x: x > 1])

263

In [35]:
len(autores_df['PAIS_EJEMPLAR'].value_counts())

127

In [36]:
len(autores_df['PAIS_EJEMPLAR'].value_counts()[lambda x: x > 1])

103

# Mapping countries

In [26]:
countries = set(autores_df['PAIS_EJEMPLAR']).union(autores_df['PAIS_AUTOR'])

In [30]:
mappings = defaultdict(set)
for a in countries:
    for b in countries:
        if a != b:
            if a in b:
                mappings[a].add(b)
mappings = dict(mappings)

In [31]:
mappings

{'Austria': {'Austria-Hungary', 'Austrian Empire', 'First Austrian Republic'},
 'Azerbaijan': {'Azerbaijan Soviet Socialist Republic'},
 'Brazil': {'Empire of Brazil'},
 'Bulgaria': {'Kingdom of Bulgaria', "People's Republic of Bulgaria"},
 'China': {'China (Republic : 1949- )',
  "People's Republic of China",
  'Republic of China (1912–49)'},
 'Colombia': {'Gran Colombia'},
 'Croatia': {'Kingdom of Croatia-Slavonia'},
 'Cuba': {'Republic of Cuba (1902–59)'},
 'Denmark': {'Kingdom of Denmark'},
 'Dominica': {'Dominican Republic'},
 'Egipto': {'Jedivato de Egipto'},
 'Finland': {'Grand Duchy of Finland'},
 'France': {'Primera República Francesa'},
 'Georgia': {'Georgia (Republic)'},
 'German Confederation': {'North German Confederation'},
 'Germany': {'Allied-occupied Germany',
  'East Germany',
  'Germany (East)',
  'Nazi Germany',
  'West Germany'},
 'Greece': {'Kingdom of Greece'},
 'Guinea': {'Equatorial Guinea', 'Papua New Guinea'},
 'Hungary': {'Austria-Hungary',
  'Kingdom of Hun

In [41]:
mappings_inv = {}
for c1 in mappings:
    for c2 in mappings[c1]:
        mappings_inv[c2] = c1

In [42]:
mappings_inv

{'Allied-occupied Germany': 'Germany',
 'Austria-Hungary': 'Hungary',
 'Austrian Empire': 'Austria',
 'Azerbaijan Soviet Socialist Republic': 'Azerbaijan',
 'Caribbean Netherlands': 'Netherlands',
 'China (Republic : 1949- )': 'China',
 'Colony of Virginia': 'Virginia',
 'Congress Poland': 'Poland',
 'Crown Colony of Malta': 'Malta',
 'Democratic Republic of the Congo': 'Republic of the Congo',
 'Dominican Republic': 'Dominica',
 'Dominion of India': 'India',
 'Dominion of New Zealand': 'New Zealand',
 'Dominion of Pakistan': 'Pakistan',
 'Duchy of Württemberg': 'Württemberg',
 'East Germany': 'Germany',
 'East Pakistan': 'Pakistan',
 'Electorate of Württemberg': 'Württemberg',
 'Empire of Brazil': 'Brazil',
 'Empire of Japan': 'Japan',
 'Equatorial Guinea': 'Guinea',
 'First Austrian Republic': 'Austria',
 'Francoist Spain': 'Spain',
 'French protectorate in Morocco': 'Morocco',
 'French protectorate of Tunisia': 'Tunisia',
 'Georgia (Republic)': 'Georgia',
 'Germany (East)': 'Germany

In [ ]:
# Remove wrong mappings
del(mappings_inv['United States of the Ionian Islands'])
# ...

# Obras por autor

In [ ]:
autores_df

In [15]:
autores_df.groupby('AUTOR')['TITULO'].count().size

21055

In [66]:
obras_por_autor_df = autores_df.set_index('AUTOR')

In [67]:
obras_por_autor_df.shape

(70687, 3)

In [68]:
obras_por_autor_df.head()

,TITULO,PAIS_EJEMPLAR,PAIS_AUTOR
AUTOR,,,
Martina Badstuber,Adivina en qué pais ...,Spain,Germany
Antón Fortes Torres,Humo,Spain,Spain
Helen Cooper,Galleta para perros,Spain,United Kingdom
Cyril Pedrosa,Autobio,Spain,France
Fina Casalderrey,"Félix, : el coleccionista de miedos",Spain,Spain


In [69]:
obras_por_autor_df.ix['Aaron Copland']

,TITULO,PAIS_EJEMPLAR,PAIS_AUTOR
AUTOR,,,
Aaron Copland,Cómo escuchar la música,Colombia,United States of America
Aaron Copland,A Copland celebration chamber music and rariti...,United States,United States of America
Aaron Copland,A Copland celebration famous orchestral and ch...,United States,United States of America
Aaron Copland,A Copland celebration vocal and choral works v...,United States,United States of America
Aaron Copland,Appalachian spring (suite),United Kingdom,United States of America
Aaron Copland,Symphony no.3 (1944-46),United States,United States of America
Aaron Copland,Clarinet Concerto,United States,United States of America
Aaron Copland,Rodeo,United States,United States of America
Aaron Copland,John Henry a railroad ballad for Orchestra,United States,United States of America


In [70]:
obras_por_autor_df.ix['A. A. Attanasio']

TITULO                El lobo y la corona
PAIS_EJEMPLAR                       Spain
PAIS_AUTOR       United States of America
Name: A. A. Attanasio, dtype: object

In [2]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{ ?author wdt:P106 wd:Q36180 . }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{ ?author rdf:type dbo:Writer . }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

def choose(opt_list):
    return Counter(opt_list).most_common(1)[0][0]

# def get_pais_autor(name):
#     r = 'DESCONOCIDO'
#     opt_dict = {}
#     for lang in ['en', 'es']:
#         opt_dict[lang] = {}
#         for db in ['dbp', 'wdt']:
#             opt_dict[lang][db] = get_pais_autor_data(db, name, lang)
#         opts = opt_dict[lang].values()
#         if all(map(lambda x: x != 'UNKOWN', opts)):
#             r = choose(opts)
#             break
#     if r == 'DESCONOCIDO':
#         for lang in ['en', 'es']:
#             opts = opt_dict[lang].values()
#             if any(map(lambda x: x != 'UNKOWN', opts)):
#                 opts = [opt for opt in opts if opt != 'UNKOWN']
#                 r = choose(opts)
#                 break
#     return opt_dict, r

def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['dbp', 'wdt']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [3]:
get_pais_autor('Mario Gómez Torrente')

'DESCONOCIDO'

In [83]:
get_pais_autor('Aristóteles')

'Grecia'

In [84]:
get_pais_autor('Miguel de Cervantes')

'Spain'

In [85]:
get_pais_autor('Aristóteles')

'Grecia'

In [54]:
autores_df[autores_df['PAIS_AUTOR'] == 'DESCONOCIDO']['AUTOR'].nunique()

122892

In [55]:
prestamos_df['AUTOR'].nunique()

148860

In [57]:
148860 - 122892

25968